In [38]:
# tools
import boto3
from mypy_boto3_bedrock_runtime.type_defs import ToolConfigurationTypeDef

RESPONSE_SCHEMA: ToolConfigurationTypeDef = {
    "tools": [
        {
            "toolSpec": {
                "name": "separate_questions_and_answers",
                "description": "質問と回答を分離する",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "question": {"type": "string"},
                            "answer": {"type": "string"},
                        },
                        "required": ["question", "answer"],
                    }
                }
            }
        }
    ]
}


def make_question_and_ground_truth(
    region: str,
    model_id: str = "us.meta.llama4-maverick-17b-instruct-v1:0"
):
    runtime_client = boto3.client("bedrock-runtime", region_name=region)
    prompt = """東京都への観光客がしそうな質問とそれに対する模範回答を考えてください。ただし質問は<question></question>、模範回答は<answer></answer>で囲んでください。"""
    try:
        response = runtime_client.converse(
            modelId=model_id,
            messages=[{"role": "user", "content": [{"text": prompt}]}]
        )
    except Exception as e:
        print(f"予期せぬエラーが発生しました: {e}")
        raise
    
    message = response["output"].get("message")
    if message:
        res = message["content"][0].get("text")
        if res:
            return res
        else:
            raise ValueError(f"{model_id}からのレスポンスに text が含まれませんでした: {message}")
    else:
        raise ValueError(f"{model_id} からのレスポンスに message が含まれませんでした: {response}")

In [39]:
import os
from dotenv import load_dotenv

load_dotenv("./.env")

REGION = os.environ.get("REGION")
if REGION:
    qa = make_question_and_ground_truth(region=REGION)
else:
    raise ValueError(f"環境変数を読み取れませんでした。\nREGION: {REGION}")



In [40]:
from pprint import pprint

pprint(qa)

('<question>東京の人気の観光スポットはどこですか？</question>\n'
 '<answer>東京の人気の観光スポットとしては、浅草の雷門や渋谷のスクランブル交差点、東京スカイツリーなどが挙げられます。浅草は歴史ある寺院や伝統的な日本の文化を体験できることで知られ、渋谷は若者のファッションやエンターテイメントの中心地として人気です。また、東京スカイツリーからは東京の街を一望することができます。</answer>\n'
 '\n'
 '<question>東京の交通手段は何が便利ですか？</question>\n'
 '<answer>東京の交通手段としては、JRや私鉄の鉄道網、都営地下鉄、バスなどが便利です。特に、SuicaやPASMOなどのICカードを利用することで、スムーズに移動することができます。また、タクシーも利用できますが、交通渋滞に巻き込まれる可能性があるため、ラッシュアワーには注意が必要です。</answer>\n'
 '\n'
 '<question>東京でおすすめの食事はなんですか？</question>\n'
 '<answer>東京では様々な料理を楽しむことができます。おすすめの食事としては、寿司、天ぷら、ラーメン、とんかつなどが挙げられます。特に、新鮮な魚介類を使った寿司は人気が高く、築地市場や銀座の寿司屋で楽しむことができます。また、ラーメンは東京の各所で様々な味を楽しむことができ、とんかつはサクサクの衣とジューシーな肉が楽しめます。</answer>\n'
 '\n'
 '<question>東京での買い物におすすめの場所はどこですか？</question>\n'
 '<answer>東京での買い物におすすめの場所としては、銀座、渋谷、新宿、原宿などが挙げられます。銀座は高級ブランドの路面店が立ち並び、渋谷は若者向けのファッションアイテムが豊富です。新宿は大規模なデパートが複数あり、原宿は個性的なファッションアイテムや雑貨が楽しめます。また、浅草の仲見世通りでは伝統的な日本の工芸品やお土産を購入することができます。</answer>\n'
 '\n'
 '<question>東京で楽しめる夜のエンターテイメントはなんですか？</question>\n'
 '<answer>東京で楽しめる夜のエンターテイメントとしては、新

In [43]:
import re

pattern = r"<question>(.*?)</question>\n<answer>(.*?)</answer>"
matches = re.findall(pattern, qa, re.DOTALL)

In [45]:
qa = {
    "question": [m[0] for m in matches],
    "answer": [m[1] for m in matches]
}

In [46]:
qa

{'question': ['東京の人気の観光スポットはどこですか？',
  '東京の交通手段は何が便利ですか？',
  '東京でおすすめの食事はなんですか？',
  '東京での買い物におすすめの場所はどこですか？',
  '東京で楽しめる夜のエンターテイメントはなんですか？'],
 'answer': ['東京の人気の観光スポットとしては、浅草の雷門や渋谷のスクランブル交差点、東京スカイツリーなどが挙げられます。浅草は歴史ある寺院や伝統的な日本の文化を体験できることで知られ、渋谷は若者のファッションやエンターテイメントの中心地として人気です。また、東京スカイツリーからは東京の街を一望することができます。',
  '東京の交通手段としては、JRや私鉄の鉄道網、都営地下鉄、バスなどが便利です。特に、SuicaやPASMOなどのICカードを利用することで、スムーズに移動することができます。また、タクシーも利用できますが、交通渋滞に巻き込まれる可能性があるため、ラッシュアワーには注意が必要です。',
  '東京では様々な料理を楽しむことができます。おすすめの食事としては、寿司、天ぷら、ラーメン、とんかつなどが挙げられます。特に、新鮮な魚介類を使った寿司は人気が高く、築地市場や銀座の寿司屋で楽しむことができます。また、ラーメンは東京の各所で様々な味を楽しむことができ、とんかつはサクサクの衣とジューシーな肉が楽しめます。',
  '東京での買い物におすすめの場所としては、銀座、渋谷、新宿、原宿などが挙げられます。銀座は高級ブランドの路面店が立ち並び、渋谷は若者向けのファッションアイテムが豊富です。新宿は大規模なデパートが複数あり、原宿は個性的なファッションアイテムや雑貨が楽しめます。また、浅草の仲見世通りでは伝統的な日本の工芸品やお土産を購入することができます。',
  '東京で楽しめる夜のエンターテイメントとしては、新宿の歌舞伎町や六本木のクラブ、ディスコなどが挙げられます。また、浅草の花やしきや東京ドームシティの遊園地も夜に楽しむことができます。さらに、東京タワーや東京スカイツリーの夜景もロマンチックでおすすめです。']}

In [47]:
from backend import call_rag

knowledge_base_id = os.environ.get("KNOWLEDGE_BASE_ID")
responses = {
    "rag_answer": [],
    "retrieved_context": []
}
if knowledge_base_id:
    for question in qa["question"]:
        res = call_rag(question=question, knowledge_base_id=knowledge_base_id, region=REGION)
        responses["rag_answer"].append(res["answer"])
        responses["retrieved_context"].append(res["context"])
else:
    print("環境変数を読み取れませんでした。")

In [48]:
responses

{'rag_answer': ['わかりません',
  '東京の交通手段としては、東京モノレールや京急電鉄、リムジンバス、タクシーが便利です。',
  'わかりません',
  'この情報だけではよくわかりません。',
  'わかりません'],
 'retrieved_context': ['また、主に観光目的で東京湾から隅田川にかけての航路が整備されているが、旧東京市内の東部に広がっていた河川や運河の水路網は戦後復興におけるがれき処理などで大半が埋め立てられ、東京都内での内水路利用は大阪と比較すると非常に限定的である。 ## 情報・メディア \\[編集\\]\n===\n?title=%E6%9D%B1%E4%BA%AC%E9%83%BD%E6%B8%8B%E8%B0%B7%E5%85%AC%E5%9C%92%E9%80%9A%E3%82%8A%E3%82%AE%E3%83%A3%E3%83%A9%E3%83%AA%E3%83%BC&action=edit&redlink=1)、[東京文化会館](https://ja.wikipedia.org/wiki/%E6%9D%B1%E4%BA%AC%E6%96%87%E5%8C%96%E4%BC%9A%E9%A4%A8)、[東京芸術劇場](https://ja.wikipedia.org/wiki/%E6%9D%B1%E4%BA%AC%E8%8A%B8%E8%A1%93%E5%8A%87%E5%A0%B4)などがある[\\[79\\]。 ### 観光 \\[編集\\] ![]() 正福寺 地蔵堂（東村山市 国宝）（2010年6月11日撮影\n===\n東京都が運営する文化施設には、東京都江戸東京博物館](https://ja.wikipedia.org/wiki/%E6%9D%B1%E4%BA%AC%E9%83%BD%E6%B1%9F%E6%88%B8%E6%9D%B1%E4%BA%AC%E5%8D%9A%E7%89%A9%E9%A4%A8)、[江戸東京たてもの園](https://ja.wikipedia.org/wiki/%E6%B1%9F%E6%88%B8%E6%9D%B1%E4%BA%AC%E3%81%9F%E3%81%A6%E3%82%82%E3%81%AE%E5%9C%92)、[東京都庭園美術館](https:

In [50]:
import pandas as pd

merged = qa | responses
df = pd.DataFrame(merged)

In [51]:
df

,question,answer,rag_answer,retrieved_context
0,東京の人気の観光スポットはどこですか？,東京の人気の観光スポットとしては、浅草の雷門や渋谷のスクランブル交差点、東京スカイツリーなど...,わかりません,また、主に観光目的で東京湾から隅田川にかけての航路が整備されているが、旧東京市内の東部に広が...
1,東京の交通手段は何が便利ですか？,東京の交通手段としては、JRや私鉄の鉄道網、都営地下鉄、バスなどが便利です。特に、Suica...,東京の交通手段としては、東京モノレールや京急電鉄、リムジンバス、タクシーが便利です。,都心部との交通手段として[東京モノレール](https://ja.wikipedia.org...
2,東京でおすすめの食事はなんですか？,東京では様々な料理を楽しむことができます。おすすめの食事としては、寿司、天ぷら、ラーメン、と...,わかりません,なお、東京都では高齢者向けの福祉事業として東京都シルバーパス](https://ja.wik...
3,東京での買い物におすすめの場所はどこですか？,東京での買い物におすすめの場所としては、銀座、渋谷、新宿、原宿などが挙げられます。銀座は高級...,この情報だけではよくわかりません。,16. **^](https://ja.wikipedia.org/wiki/%E6%9D%...
4,東京で楽しめる夜のエンターテイメントはなんですか？,東京で楽しめる夜のエンターテイメントとしては、新宿の歌舞伎町や六本木のクラブ、ディスコなどが...,わかりません,16. **^](https://ja.wikipedia.org/wiki/%E6%9D%...
